In [10]:
##test  
import spacy

In [37]:
import spacy
from typing import List, Optional
from pydantic import BaseModel

class Ingredient(BaseModel):
    name: str
    description: str
    quantity: float
    measurement_unit: str

class Product(BaseModel):
    id: str
    name: str
    image_url: str
    quantity: float
    measurement_unit: str
    description: str
    category: str
    price: float
    reviews: Optional[int]
    rating: Optional[int]

class Recipe(BaseModel):
    id: str
    name: str
    image_url: str
    category: str
    steps: List[str]
    ingredients: List[Ingredient]
    reviews: Optional[int]
    rating: Optional[int]


def find_matching_ingredients(recipe: Recipe, products: List[Product], similarity_threshold: float = 0.7):
    nlp = spacy.load("ro_core_news_md")
    
    matching_ingredients = []
    
    for recipe_ingredient in recipe.ingredients:
        recipe_ingredient_tokens = nlp(recipe_ingredient.name)
        recipe_ingredient_matches = []
        
        for product in products:
            product_tokens = nlp(product.name)
            
            similarity = recipe_ingredient_tokens.similarity(product_tokens)
            
            if similarity >= similarity_threshold:
                recipe_ingredient_matches.append((product, similarity))
        
        recipe_ingredient_matches = sorted(recipe_ingredient_matches, key=lambda x: x[1], reverse=True)
        
        if len(recipe_ingredient_matches) > 0:
            closest_quantity_product = None
            closest_quantity_diff = float('inf')
            
            for product, similarity in recipe_ingredient_matches:
                if product.quantity > recipe_ingredient.quantity:
                    quantity_diff = abs(product.quantity - recipe_ingredient.quantity)
                    
                    if quantity_diff < closest_quantity_diff:
                        closest_quantity_product = product
                        closest_quantity_diff = quantity_diff
            
            if closest_quantity_product is not None:
                matching_ingredients.append((recipe_ingredient, closest_quantity_product))
    
    return matching_ingredients

# Example usage
recipes = [
    Recipe(
        id="1",
        name="Chocolate Cake",
        image_url="cake.jpg",
        category="dessert",
        steps=["Step 1", "Step 2", "Step 3"],
        ingredients=[
            Ingredient(name="făină", description="Făină albă", quantity=300, measurement_unit= "g"),
            Ingredient(name="zahăr", description="Zahăr granulat", quantity=200, measurement_unit= 'g'),
            Ingredient(name="cacao", description="Cacao neîndulcită", quantity=100, measurement_unit= 'g'),
        ],
        reviews=10,
        rating=4
    ),
    Recipe(
        id="2",
        name="Vegetable Stir-Fry",
        image_url="stir_fry.jpg",
        category="vegetarian",
        steps=["Step 1", "Step 2", "Step 3"],
        ingredients=[
            Ingredient(name="broccoli", description="Broccoli proaspăt", quantity=2, measurement_unit= 'buc'),
            Ingredient(name="morcovi", description="Morcovi feliați", quantity=2, measurement_unit= 'buc'),
            Ingredient(name="tofu", description="Tofu tare, cuburi", quantity=50, measurement_unit= 'g'),
        ],
        reviews=5,
        rating=3
    )
]

products = [
    Product(
        id="1",
        name="făină albă",
        image_url="flour.jpg",
        quantity="1000",
        measurement_unit="g",
        description="Făină versatilă pentru copt și gătit",
        category="păntry",
        price=2.99,
        reviews=100,
        rating=4
    ),
    Product(
        id="2",
        name="zahăr granulat",
        image_url="sugar.jpg",
        quantity="500",
        measurement_unit="g",
        description="Îndulcitor comun pentru băuturi și deserturi",
        category="păntry",
        price=5.5,
        reviews=100,
        rating=4
    ), 
    Product(
        id="3",
        name="cacao neîndulcită",
        image_url="cocoa.jpg",
        quantity="200",
        measurement_unit="g",
        description="Cacao naturală pentru prăjituri și băuturi calde",
        category="păntry",
        price=3.49,
        reviews=80,
        rating=4
    ),
    Product(
        id="4",
        name="ulei de măsline extra virgin",
        image_url="olive_oil.jpg",
        quantity="500",
        measurement_unit="ml",
        description="Ulei de măsline presat la rece, de calitate superioară",
        category="păntry",
        price=7.99,
        reviews=120,
        rating=5
    ),
    Product(
        id="5",
        name="ciocolată neagră 70%",
        image_url="dark_chocolate.jpg",
        quantity="100",
        measurement_unit="g",
        description="Ciocolată neagră cu conținut ridicat de cacao",
        category="dulciuri",
        price=2.49,
        reviews=90,
        rating=4
    ),
    Product(
        id="6",
        name="ardei gras roșu",
        image_url="red_pepper.jpg",
        quantity="1",
        measurement_unit="buc",
        description="Ardei gras roșu, proaspăt și aromat",
        category="fructe & legume",
        price=1.29,
        reviews=50,
        rating=4
    ),
    Product(
        id="7",
        name="zahăr pudra",
        image_url="sugar.jpg",
        quantity="300",
        measurement_unit="g",
        description="Îndulcitor comun pentru băuturi și deserturi",
        category="păntry",
        price=5.5,
        reviews=100,
        rating=4
    ),
]


       


In [38]:
find_matching_ingredients(recipes[0],products)

[(Ingredient(name='făină', description='Făină albă', quantity=300.0, measurement_unit='g'),
  Product(id='1', name='făină albă', image_url='flour.jpg', quantity=1000.0, measurement_unit='g', description='Făină versatilă pentru copt și gătit', category='păntry', price=2.99, reviews=100, rating=4)),
 (Ingredient(name='zahăr', description='Zahăr granulat', quantity=200.0, measurement_unit='g'),
  Product(id='7', name='zahăr pudra', image_url='sugar.jpg', quantity=300.0, measurement_unit='g', description='Îndulcitor comun pentru băuturi și deserturi', category='păntry', price=5.5, reviews=100, rating=4)),
 (Ingredient(name='cacao', description='Cacao neîndulcită', quantity=100.0, measurement_unit='g'),
  Product(id='3', name='cacao neîndulcită', image_url='cocoa.jpg', quantity=200.0, measurement_unit='g', description='Cacao naturală pentru prăjituri și băuturi calde', category='păntry', price=3.49, reviews=80, rating=4))]